In [1]:
using JuMP, Ipopt, Complementarity
I = 1:4
J = 1:2
K = 1:3
L = 1:4
x0 = [1, 1, -1, -1]
y0 = zeros(length(J), length(K))
ll0 = ones(length(L), length(K))
model = Model(Ipopt.Optimizer)
@variable(model, x[i in I], start = x0[i])
@variable(model, y[j in J, k in K], start = y0[j, k])
@variable(model, ll[l in L, k in K], start = ll0[l, k])
@objective(model, Max, (x[1] - x[3]) * (x[2] - x[4]))
@constraints(model, begin
    -y[1,1] - y[2,1]^2  <= 0
    y[1,2]/4 + y[2,2] - 3/4 <= 0
    -y[2,3] - 1 <= 0
    1        + ll[1,1] - ll[3,1] == 0
    2*y[2,1] + ll[2,1] - ll[4,1] == 0
    -1/4     + ll[1,2] - ll[3,2] == 0
     -1      + ll[2,2] - ll[4,2] == 0
    0        + ll[1,3] - ll[3,3] == 0
    1        + ll[2,3] - ll[4,3] == 0
end)
for k in K
    @complements(model, 0 >=   y[1,k] - x[1], ll[1,k] >= 0)
    @complements(model, 0 >=   y[2,k] - x[2], ll[2,k] >= 0)
    @complements(model, 0 >= - y[1,k] + x[3], ll[3,k] >= 0)
    @complements(model, 0 >= - y[2,k] + x[4], ll[4,k] >= 0)
end
optimize!(model)
solution_summary(model)
value.(x)
value.(y)
value.(ll)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:       49
Number of nonzeros in inequality constraint Jacobian.:       41
Number of nonzeros in Lagrangian Hessian.............:       77

Total number of variables............................:       22
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       18
Total number of inequality c

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:4
    Dimension 2, 1:3
And data, a 4×3 Matrix{Float64}:
 216.269      13978.7     22454.7
   6.19579e5     96.0735     95.0755
 217.269      13978.4     22454.7
   6.19578e5     95.0735     96.0755

In [2]:
using JuMP, Ipopt, Complementarity

# Crear el modelo
model = Model(Ipopt.Optimizer)
# Definir el conjunto I
I = 1:5

# Definir las variables
@variable(model, y1)
@variable(model, y2)
@variable(model, x >= 0)
@variable(model, s[i in I] >= 0)  # Variables s[i] no negativas
@variable(model, l[i in I] >= 0)  # Variables l[i] no negativas

# Definir la función objetivo
@objective(model, Min, -x - 3*y1 + 2*y2)

# Agregar restricciones del problema interno
@constraint(model, c1, -2*x + y1 + 4*y2 + s[1] == 16)
@constraint(model, c2, 8*x + 3*y1 - 2*y2 + s[2] == 48)
@constraint(model, c3, -2*x + y1 - 3*y2 + s[3] == -12)
@constraint(model, c4, -y1 + s[4] == 0)
@constraint(model, c5, y1 + s[5] == 4)

# Agregar condiciones KKT para el óptimo del problema interno
@constraint(model, kt1, -1 + l[1] + 3*l[2] + l[3] - l[4] + l[5] == 0)
@constraint(model, kt2, 4*l[2] - 2*l[2] - 3*l[3] == 0)

# Restricciones de complementariedad

for i in I
    @complements(model, 0<=l[i],s[i]>=0) # Complementariedad entre l[i] y s[i]
end

# Resolver el modelo
optimize!(model)

# Mostrar resultados
println("Estado de la solución: ", termination_status(model))
println("Valor óptimo de x: ", value(x))
println("Valor óptimo de y1: ", value(y1))
println("Valor óptimo de y2: ", value(y2))
println("Valores óptimos de s: ", value.(s))
println("Valores óptimos de l: ", value.(l))


This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:       33
Number of nonzeros in inequality constraint Jacobian.:       10
Number of nonzeros in Lagrangian Hessian.............:       15

Total number of variables............................:       13
                     variables with only lower bounds:       11
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       12
Total number of inequality constraints...............:       10
        inequality constraints with only lower bounds:       10
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -9.9999900e-03 4.79e+01 2.80e-03  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [28]:
using JuMP, Ipopt, Complementarity

# Crear el modelo
model = Model(Ipopt.Optimizer)
@variable(model,0<=x[1:2]<=50)
@variable(model,y[1:2])
@variable(model,0<=l[1:6])
@objective(model,Min,2x[1]+2x[2]-3y[1]-3y[2]-60)
@constraints(model,begin
x[1]+x[2]+y[1]-2y[2]-40<=0
2y[1]-2x[1]+40-(l[1]-l[2]-2l[5])==0
2y[2]-2x[2]+40-(l[3]-l[4]-2l[6])==0

end)




(x[1] + x[2] + y[1] - 2 y[2] <= 40, -2 x[1] + 2 y[1] - l[1] + l[2] + 2 l[5] == -40, -2 x[2] + 2 y[2] - l[3] + l[4] + 2 l[6] == -40)

In [29]:
@complements(model,0<=y[1]+10,l[1]>=0)

(sqrt(l[1] ^ 2.0 + (10.0 + y[1]) ^ 2.0 + 1.0e-8) - (l[1] + (10.0 + y[1]))) - 0.0 == 0

In [30]:
@complements(model,
0 <= -y[1] + 20,           l[2] >= 0


)

(sqrt(l[2] ^ 2.0 + (20.0 + -1.0 * y[1]) ^ 2.0 + 1.0e-8) - (l[2] + (20.0 + -1.0 * y[1]))) - 0.0 == 0

In [31]:
@complements(model,0<=y[2]+10,l[3]>=0)
@complements(model,0<=-y[2]+20,l[4]>=0)
@complements(model,0<=x[1]-2y[1]-10,l[5]>=0)
@complements(model,0<=x[2]-2y[2]-10,l[6]>=0)


(sqrt(l[6] ^ 2.0 + (-10.0 + x[2] + -2.0 * y[2]) ^ 2.0 + 1.0e-8) - (l[6] + (-10.0 + x[2] + -2.0 * y[2]))) - 0.0 == 0

In [32]:
optimize!(model)
#solution_summary(model)


This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:       24
Number of nonzeros in inequality constraint Jacobian.:       18
Number of nonzeros in Lagrangian Hessian.............:       24

Total number of variables............................:       10
                     variables with only lower bounds:        6
                variables with lower and upper bounds:        2
                     variables with only upper bounds:        0
Total number of equality constraints.................:        8
Total number of inequality constraints...............:       13
        inequality constraints with only lower bounds:       12
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        1

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -5.9960000e+01 4.00e+01 4.05e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [34]:
value.(x)
value.(y)
value.(l)

6-element Vector{Float64}:
 3.333346297150919e-10
 3.3333281843553245e-10
 2.5000069959560875e-10
 5.000014825635611e-10
 9.999935658919406e-10
 8.660379107315982e-5

In [35]:
solution_summary(model)

* Solver : Ipopt

* Status
  Result count       : 1
  Termination status : LOCALLY_SOLVED
  Message from the solver:
  "Solve_Succeeded"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 5.00017e+00
  Dual objective value : -1.00000e+01

* Work counters
  Solve time (sec)   : 3.79999e-02
  Barrier iterations : 35
